<div align='center'>
    <p></p>
<h3>Web Time Series Service (WTSS)</h3>
    <p></p>
</div>

<div align='justify'>
    
    Este notebook contém uma adaptação do serviço WTSS. Neste script o usuário fornece o nome de um município e do estado no qual esse município está localizado, um intervalo temporal, um ou mais atributos e um método de agregação. A partir disso, são obtidas as séries temporais filtradas seguindo esses parâmetros. Por fim, o usuário pode apresentar graficamente um mapa com os resultados obtidos.

    
<h4>Obter as Localizações</h4>    

    A partir da escolha do município, o script busca nos shapefiles disponíveis (que são as malhas municipais disponibilizadas pelo IBGE) o polígono que o delimita, aplica uma grade com resolução definida pelo usuário e obtém as localizações para consultar o serviço. Logo abaixo há um exemplo que demonstra esse processo.
    
<img src="./Images/obtaing_points.png" width=80% heigth=60%> 
    
<h4>Teste das grades</h4>
    
    A resolução da grade definida está relacionada a quantidade de pontos desejados para consultar as séries temporais de uma região. Quanto maior o valor da resolução de x e y, mais pontos serão obtidos. Por exemplo, uma grade de 10x10 retornará menos pixels do que uma grade de 150x150. 
    Alguns testes foram feitos para medir a velocidade desta aborgadem e verificar se o WTSS responderia bem a ela. Esses testes foram:
    
<ul>
    <li> <b>Região</b>: Cametá-PA
    <li> <b>Período</b>: 2020-01-01 a 2020-12-31
    <li> <b>Cubo</b>: MOD13Q1-6
    <li> <b>Atributos</b>: 'EVI' e 'NDVI'
    <li> <b>Grades</b>: 
        <ul><b>10x10</b>:
            <li>Tempo de execução: 15s
            <li>Pontos: 1242
            <li>Threads: 4
        </ul>
        <ul><b>20x20</b>:
            <li>Tempo de execução: 47s
            <li>Pontos: 5083
            <li>Threads: 4
        </ul>
        <ul><b>30x30</b>:
            <li>Tempo de execução: 1min 27s
            <li>Pontos: 11316
            <li>Threads: 4
        </ul>
        <ul><b>40x40</b>:
            <li>Tempo de execução: 2min e 37s
            <li>Pontos: 20148
            <li>Threads: 4
        </ul>
        <ul><b>50x50</b>:
            <li>Tempo de execução: 4min e 57s
            <li>Pontos: 31556
            <li>Threads: 4
        </ul>
        <ul><b>60x60</b>:
            <li>Tempo de execução: 7min e 15s
            <li>Pontos: 45218
            <li>Threads: 4
        </ul>
        <ul><b>80x80</b>:
            <li>Tempo de execução: 11min e 05s
            <li>Pontos: 80592
            <li>Threads: 4
        </ul>
        <ul><b>100x100</b>:
            <li>Tempo de execução: 17min e 50s
            <li>Pontos: 125833
            <li>Threads: 4
        </ul>
        <ul><b>120x120</b>:
            <li>Tempo de execução: 28min e 14s
            <li>Pontos: 181079
            <li>Threads: 4
        </ul>
        <ul><b>150x150</b>:
            <li>Tempo de execução: 43min
            <li>Pontos: 283222
            <li>Threads: 4
        </ul>
</ul>
    
</div>

<h4>Instalação e importação das bibliotecas</h4>

In [1]:
#!pip install --upgrade pip setuptools wheel
#!pip install numpy
#!pip install geopandas
#!pip install pandas
#!pip install wtss
#!pip install matplotlib
#!pip install mapclassify
#!pip install folium
#!pip install shapely

In [1]:
import wtss_harmonize

<div align='justify'>
    
A célula abaixo instancia um objeto que permite acesso as seguintes funcionalidades:

<ul>
    <li> <b>list_coverages:</b> lista os cubos disponíveis no repositório do BDC;
    <li> <b>list_coverage_attributes:</b> lista os atributos dos cubos (ex: NDVI, EVI etc);
    <li> <b>get_timeseries:</b> obtém as séries temporais para uma determinada região;
    <li> <b>plot:</b> apresenta graficamente as séries temporais;
</ul>

Para instanciar este objeto, é necessário informar um <i>token</i> de acesso, que é gerado pelo BDC para utilizar as ferramentas disponibilizadas pelo projeto.
    
</div>

In [2]:
service = wtss_harmonize.WTSS_Harmonize(token='access_token')

In [3]:
service.list_coverages

['MOD13Q1-6',
 'MYD13Q1-6',
 'S2-SEN2COR_10_16D_STK-1',
 'LC8_30_16D_STK-1',
 'CB4MUX_20_1M_STK-1',
 'LC8_30_6M_MEDSTK-1',
 'CB4_64_16D_STK-1',
 'LANDSAT-MOZ_30_1M_STK-1',
 'S2-16D-2',
 'CBERS-WFI_64_8D_LCF-1']

In [4]:
service.list_coverage_attributes(name='CBERS-WFI_64_8D_LCF-1')

['BAND13',
 'BAND14',
 'BAND15',
 'BAND16',
 'CMASK',
 'EVI',
 'NDVI',
 'CLEAROB',
 'TOTALOB',
 'PROVENANCE',
 'DATASOURCE']

O método <b>get_timeseries</b> necessita dos seguinte parâmetros:

<ul>
    <li><b>município</b>: é uma <i>string</i> que representa o nome do município definido para obter as séries temporais;
    <li><b>uf</b>: é uma <i>string</i> que representa a abreviação dos estados;
    <li><b>start_date</b>: é uma <i>string</i> que marca o início do período de obtenção das séries;
    <li><b>end_date</b>: é uma <i>string</i> que marca o término do período de obtenção das séries;
    <li><b>coverage</b>: é uma <i>string</i> que define o nome do cubo para consulta das séries temporais;
    <li><b>attributes</b>: é uma <i>tupla</i> de <i>strings</i> que aponta os atributos a serem consultados;
    <li><b>grid</b>: é uma <i>tupla</i> de <i>inteiros</i> que aponta a resolução da grade utilizada para obter as localizações para consultar as séries temporais;
</ul>

In [5]:
x, y = 100, 100
df_ts = service.get_timeseries(municipio='cametá', uf='pa', start_date='2020-01-01', end_date='2020-12-31', 
                               coverage='MOD13Q1-6', attributes=('EVI', 'NDVI'), grid=(x, y))

O método <b>plot</b> necessita dos seguinte parâmetros:

<ul>
    <li><b>df</b>: é uma <i>geoDataFrame</i> que contém os valores das séries consultadas;
    <li><b>attributes</b>: é uma <i>lista</i> que contém os atributos a serem apresentados;
    <li><b>aggregation</b>: é uma <i>string</i> que define o método de agregação aplicada nos dados;
    <li><b>start_date</b>: é uma <i>string</i> que define a data de início da representação gráfica;
    <li><b>end_date</b>: é uma <i>string</i> que define a data de término da representação gráfica. Caso o usuário não defina uma data de término, é necessário definir o parâmetro <i>qtd_days</i>, para aplicar a agregação dos dados para um período delimitado;
    <li><b>qtd_days</b>: é uma <i>inteiro</i> que delimita o período de agregação a ser feita. Caso não seja definido o <i>qtd_days</i>, é necesário definir o <i>end_date</i>;
</ul>

In [7]:
service.plot(df=df_ts, attributes=['EVI', 'NDVI'], aggregation='mean', start_date='2020-01-01', qtd_days=60)

/home/yuri/Documentos/teste_wtss/venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1676: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = data.groupby(**groupby_kwargs).agg(aggfunc)
/home/yuri/Documentos/teste_wtss/venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1676: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = data.groupby(**groupby_kwargs).agg(aggfunc)
